## データについて
各カラムの意味は以下の通りです.
<table>
  <thead>
  <tr>
    <td>.csvファイル</td>
    <td>学習モデル</td>
    <td>CVスコア</td>
    <td>暫定スコア</td>
    <td>備考</td>     
  </tr>
  </thead>
  <tr>
    <td>submission06102200.csv</td>
    <td>LR only</td> 
    <td>0.87363</td>
    <td>0.86669</td>
    <td>officer_idは100000のみ</td>
  </tr>
  <tr>
    <td>submission06111044.csv</td>
    <td>{LR & RF(n=100)}/2</td> 
    <td>0.87809</td>
    <td>0.?????</td>
    <td>同上</td>
  </tr>
  <tr>
    <td>submission06111100.csv</td>
    <td>{LR & RF(n=100)}/2</td> 
    <td>0.88333</td>
    <td>0.?????</td>
    <td>stop_hourを修正</td>
  </tr> 
  <tr>
    <td>なし</td>
    <td>XGB(深さ5)</td> 
    <td>0.88728</td>
    <td>0.?????</td>
    <td></td>
  </tr>  
  <tr>
    <td>なし</td>
    <td>XGB(深さ4,n_est=200)</td> 
    <td>0.88848</td>
    <td>0.?????</td>
    <td></td>
  </tr>   
  <tr>
    <td>なし</td>
    <td>XGB(深さ5)とLRを10:1</td> 
    <td>0.88766</td>
    <td>0.?????</td>
    <td></td>
  </tr>  
  <tr>
    <td>なし</td>
    <td>XGB(深さ4)</td> 
    <td>0.86897</td>
    <td>0.?????</td>
    <td>officer_idを追加した</td>
  </tr>  
  <tr>
    <td>submission_06111430.csv</td>
    <td>XGB(確か4と400)</td> 
    <td>0.?????</td>
    <td>0.?????</td>
    <td>nyx1で実行</td>
  </tr>   
  <tr>
    <td>submission_06111430.csv</td>
    <td>XGB&LR&RF</td> 
    <td>0.88907</td>
    <td>0.?????</td>
    <td>nyx1で実行</td>
  </tr>  
  <tr>
    <td>submission_06131630.csv</td>
    <td>xgb(4,200):mlp:lgb=70:70:3</td> 
    <td>0.8869</td>
    <td>0.?????</td>
    <td>nyx1で実行</td>
  </tr>     
</table>


In [104]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from datetime import datetime

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
#m = xgb.XGBRegressor()
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures
from lightgbm import LGBMClassifier

In [21]:
df = pd.read_csv("train_clean_06111230.csv")
df_test = pd.read_csv("test_clean_06111230.csv")

In [2]:
df = pd.read_csv("train_clean_06131000.csv")
df_test = pd.read_csv("test_clean_06131000.csv")

In [18]:
df = pd.read_csv("train_clean_06131430.csv")
df_test = pd.read_csv("test_clean_06131430.csv")

In [22]:
X = df.drop(["is_arrested"],axis=1)
y = df["is_arrested"]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

/home/minoda/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [38]:
%%time
lgbc = LGBMClassifier(class_weight="balanced",n_estimators=30,learning_rate=0.3
                      ,reg_lambda=40,num_leaves=45)
lgbc.fit(X_train,y_train)
y_predict_lgb = lgbc.predict_proba(X_test)[:,1]
y_train_predict_lgb = lgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_lgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_lgb))

train data score: 0.9305192812494529
cross validation score: 0.8821732308886127
CPU times: user 13.9 s, sys: 1.3 s, total: 15.2 s
Wall time: 2.18 s


In [52]:
forest = RandomForestClassifier(n_estimators=600,class_weight="balanced")
forest.fit(X_train,y_train)
y_predict_forest = forest.predict_proba(X_test)[:,1]
y_train_predict_forest = forest.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_forest))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_forest))

0.8663692679122309

In [13]:
%%time
xgbc = xgb.XGBClassifier(max_depth=4,n_estimators=500)
xgbc.fit(X_train,y_train)
y_predict_xgb = xgbc.predict_proba(X_test)[:,1]
y_train_predict_xgb = xgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_xgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_xgb))

train data score: 0.9211646354778193
cross validation score: 0.8729738390793421
CPU times: user 8min 7s, sys: 2min 10s, total: 10min 17s
Wall time: 10min 16s


In [227]:
%%time
xgbc = xgb.XGBClassifier(max_depth=4,n_estimators=200)
xgbc.fit(X_train,y_train)
y_predict_xgb = xgbc.predict_proba(X_test)[:,1]
y_train_predict_xgb = xgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_xgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_xgb))

train data score: 0.9015736824353404
cross validation score: 0.8830580869167379
CPU times: user 3min 15s, sys: 53 s, total: 4min 8s
Wall time: 4min 8s


In [ ]:
%%time
xgbc = xgb.XGBClassifier(max_depth=5,n_estimators=100)
xgbc.fit(X_train,y_train)
y_predict_xgb = xgbc.predict_proba(X_test)[:,1]
y_train_predict_xgb = xgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_xgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_xgb))

<table class="table2">
    <caption>06121820</caption>
  <tr><th></th><th>3</th><th>4</th><th>5</th><th>6</th></tr>
  <tr><td>100</td><td>???,???</td><td>??? / ???</td><td>0.8990 / 0.8768</td><td>???,???</td></tr>
  <tr><td>200</td><td>???,???</td><td>??? / </td><td>???,???</td><td>???,???</td></tr>
  <tr><td>300</td><td>???,???</td><td>??? / >???</td><td>???,???</td><td>???,???</td></tr>
    <tr><td>500</td><td>???? / ????</td><td>0.9181 / 0.8734</td><td>?? / ???</td><td>???,???</td></tr>
  <tr><td>700</td><td>0.9054 / 0.8726</td><td>???? / ????</td><td>??? / ???</td><td>???,???</td>
</table>


<table class="table2">
    <caption>06111230</caption>
  <tr><th></th><th>3</th><th>4</th><th>5</th><th>6</th></tr>
  <tr><td>100</td><td>???,???</td><td>0.8903 / 0.8768</td><td>??? / 0.8872?</td><td>???,???</td></tr>
  <tr><td>200</td><td>???,???</td><td>??? / </td><td>???,???</td><td>???,???</td></tr>
  <tr><td>300</td><td>???,???</td><td>0.9093 / 0.8793</td><td>???,???</td><td>???,???</td></tr>
    <tr><td>500</td><td>0.9004,0.8791</td><td>0.9184 / 0.8811</td><td>0.9376 / 0.8699</td><td>???,???</td></tr>
  <tr><td>700</td><td>???,???</td><td>0.9272 / 0.8809</td><td>??? / ???</td><td>???,???</td>
</table>

In [39]:
%%time
lr = LogisticRegression(class_weight="balanced",C=0.1)
lr.fit(X_train,y_train)
y_predict_lr = lr.predict_proba(X_test)[:,1]
y_train_predict_lr = lr.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_lr))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_lr))

train data score: 0.8823167460314494
cross validation score: 0.8749300894535248
CPU times: user 4.44 s, sys: 400 ms, total: 4.84 s
Wall time: 2.87 s


In [46]:
%%time
mlp = MLPClassifier(alpha=1e-2,hidden_layer_sizes=6, random_state=1)
mlp.fit(X_train,y_train)
y_predict_mlp = mlp.predict_proba(X_test)[:,1]
y_train_predict_mlp = mlp.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_mlp))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_mlp))

train data score: 0.8903967337372672
cross validation score: 0.8827904010101835
CPU times: user 2min 8s, sys: 5.14 s, total: 2min 13s
Wall time: 17.4 s


In [120]:
%%time
adbc = AdaBoostClassifier(n_estimators=100)
adbc.fit(X_train,y_train)
y_predict_adb = adbc.predict_proba(X_test)[:,1]
y_train_predict_adb = adbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_adb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_adb))

train data score: 0.87848785683378
cross validation score: 0.8695935235015916
CPU times: user 30.7 s, sys: 208 ms, total: 30.9 s
Wall time: 30.9 s


<table>
  <caption>clean_06121840</caption>
  <thead>
  <tr>
    <td>あるふぁ</td>
    <td>隠れ層サイズ</td>
    <td>train data score</td>  
    <td>cross validation score</td>       
  </tr>
  </thead>
  <tr>
    <td>1e-2</td>
    <td>10</td> 
    <td>0.8877</td>
    <td>0.8893</td>
  </tr>
  <tr>
    <td>1e-2</td>
    <td>6</td> 
    <td>0.8887</td>
    <td>0.8825</td>
  </tr> 
</table>
<table>
  <caption>clean_06111230</caption>
  <thead>
  <tr>
    <td>あるふぁ</td>
    <td>隠れ層サイズ</td>
    <td>train data score</td>  
    <td>cross validation score</td>       
  </tr>
  </thead>
  <tr>
    <td>1e-5</td>
    <td>50</td> 
    <td>0.96</td>
    <td>0.82</td>
  </tr> 
  <tr>
    <td>1e-5</td>
    <td>30</td> 
    <td>0.946</td>
    <td>0.8284</td>
  </tr>    
  <tr>
    <td>1e-5</td>
    <td>20</td> 
    <td>0.9258</td>
    <td>0.8543</td>
  </tr>   
  <tr>
    <td>1e-5</td>
    <td>10</td> 
    <td>0.9072</td>
    <td>0.8718</td>
  </tr> 
  <tr>
    <td>1e-5</td>
    <td>6</td> 
    <td>0.8934</td>
    <td>0.8802</td>
  </tr>   
  <tr>
    <td>1e-3</td>
    <td>6</td> 
    <td>0.8923</td>
    <td>0.8828</td>
  </tr>
  <tr>
    <td>1e-2</td>
    <td>6</td> 
    <td>0.8910</td>
    <td>0.8834</td>
  </tr>   
  <tr>
    <td>1e-5</td>
    <td>3</td> 
    <td>0.8747</td>
    <td>0.8612</td>
  </tr>   
</table>

In [232]:
#lr,xgb,mlp,lgbm
rate = np.array([0.,80,70,3.0,0])
roc_auc_score(y_test.values,np.dot(rate,np.array([y_predict_lr,y_predict_xgb
                                                  ,y_predict_mlp,y_predict_lgb,y_predict_adb]))/np.sum(rate))

0.886925058269447

In [223]:
X_final_test = df_test

In [224]:
%%time
lgbc_final = LGBMClassifier(class_weight="balanced",n_estimators=30,learning_rate=0.3
                      ,reg_lambda=40,num_leaves=45)
lgbc_final.fit(X,y)

CPU times: user 2.25 s, sys: 568 ms, total: 2.82 s
Wall time: 2.82 s


In [233]:
%%time
xgb_final = xgb.XGBClassifier(max_depth=4,n_estimators=200)
xgb_final.fit(X,y)

CPU times: user 4min 12s, sys: 1min 6s, total: 5min 18s
Wall time: 5min 21s


In [226]:
%%time
mlp_final = MLPClassifier(alpha=1e-2,hidden_layer_sizes=6, random_state=1)
mlp_final.fit(X,y)

CPU times: user 11.9 s, sys: 1.88 s, total: 13.8 s
Wall time: 13.8 s


In [234]:
y_final_predict_xgb = xgb_final.predict_proba(X_final_test)[:,1]
y_final_predict_mlp = mlp_final.predict_proba(X_final_test)[:,1]
y_final_predict_lgbc = lgbc_final.predict_proba(X_final_test)[:,1]
rate = np.array([70,70,3])
y_final_predict = np.dot(rate,np.array([y_final_predict_xgb,y_final_predict_mlp,y_final_predict_lgbc]))/np.sum(rate)

In [235]:
y_final_predict = np.round(y_final_predict,6)

In [236]:
np.average(y_final_predict)

0.029069573430218952

In [237]:
submission = pd.DataFrame(data=y_final_predict, columns=['is_arrested'])
submission.head(3)

,is_arrested
0,0.012128
1,0.024653
2,0.437279


In [238]:
submission.to_csv("submission.csv",index=False)